## Tokenization Assignment

In [4]:
from tokenizers import Tokenizer, trainers, models, normalizers, pre_tokenizers
from tokenizers.processors import BertProcessing
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tqdm import tqdm
from datasets import load_dataset
import pandas as pd

##### Step 1 : Loading dataset

In [5]:
dataset = load_dataset("wikimedia/wikipedia", "20231101.hi", split='train')
print(f"Dataset size: {len(dataset)}")
print(dataset[0])

Dataset size: 163093
{'id': '10', 'url': 'https://hi.wikipedia.org/wiki/%E0%A4%B9%E0%A4%AE%20%E0%A4%B9%E0%A5%8B%E0%A4%82%E0%A4%97%E0%A5%87%20%E0%A4%95%E0%A4%BE%E0%A4%AE%E0%A4%AF%E0%A4%BE%E0%A4%AC', 'title': 'हम होंगे कामयाब', 'text': 'हम होंगे कामयाब ( का गिरिजा कुमार माथुर द्वारा किया गया हिंदी भावानुवाद) एक प्रतिरोध गीत है। यह गीत बीसवीं सदी में नागरिक अधिकार आंदोलन का प्रधान स्वर बना। इस गीत को आमतौर पर "I\'ll Overcome Some Day" ("आई विल ओवरकम सम डे") से काव्यावतरित माना जाता है, जो चार्ल्स अल्बर्ट टिंडले द्वारा गाया गया था और जिसे 1900 में पहली बार प्रकाशित किया गया था।\n\nसन्दर्भ\nHum Honge Kamyab Lyrics \nनागरिक अधिकार आंदोलन\nदेशभक्ति के गीत\nआधार'}


##### Step 2 : Deduplication

In [6]:
df = pd.DataFrame(dataset)
df.drop_duplicates(subset=["text"], inplace=True)
print(f"After deduplication, dataset size: {len(df)}")

After deduplication, dataset size: 158516


##### Step 3 : BPE Tokenization

In [8]:
def train_bpe_tokenizer(corpus, vocab_size=32000, min_frequency=2):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])
    tokenizer.pre_tokenizer = Whitespace()
    trainer = trainers.BpeTrainer(vocab_size=vocab_size, min_frequency=min_frequency, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
    tokenizer.train_from_iterator(corpus, trainer=trainer)
    tokenizer.post_processor = BertProcessing(("[SEP]", tokenizer.token_to_id("[SEP]")), ("[CLS]", tokenizer.token_to_id("[CLS]")))
    return tokenizer

In [9]:
corpus = df["text"].tolist()

tokenizer = train_bpe_tokenizer(corpus)

tokenizer.save("hindi_bpe_tokenizer.json")

In [10]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("hindi_bpe_tokenizer.json")

sample_text = "हिंदी भाषा का मॉडल प्रशिक्षण महत्वपूर्ण है।"
encoded = tokenizer.encode(sample_text)

print("Encoded IDs:", encoded.ids)
print("Encoded Tokens:", encoded.tokens)

decoded = tokenizer.decode(encoded.ids)
print("Decoded Text:", decoded)

Encoded IDs: [2, 4479, 4256, 515, 4669, 5651, 4716, 548, 553, 3]
Encoded Tokens: ['[CLS]', 'हद', 'भष', 'क', 'मडल', 'परशकषण', 'महतवपरण', 'ह', '।', '[SEP]']
Decoded Text: हद भष क मडल परशकषण महतवपरण ह ।
